## This Notebook generates a CSV for Table 1a
The CSV has the structure of Table 1a, except for the Total row at the bottom. The presentation and the Total row as in the paper was done in another tool.

In [15]:
from pathlib import Path
import pandas as pd

In [16]:
# define data location constants
DATA_DIR = Path('..') / 'source_data'
WILD_FISH_DATA = DATA_DIR / 'all_wild_fish_lice.csv'
WILD_EVENT_DATA = DATA_DIR / 'all_wild_sample_events.csv'

OUTPUT_DIR = Path('..') / 'output' / 'Table_1a'

In [17]:
# import the data
events_df = pd.read_csv(WILD_EVENT_DATA)
events_df['event_id'] = events_df['event_id'].astype(str)
fish_df = pd.read_csv(WILD_FISH_DATA)
fish_df['event_id'] = fish_df['event_id'].astype(str)


/tmp/ipykernel_15709/468452417.py:4: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  fish_df = pd.read_csv(WILD_FISH_DATA)


In [18]:

# make a year column
events_df['sampledate'] = pd.to_datetime(events_df.sampledate, errors='raise', utc=True)
events_df['year'] = events_df['sampledate'].dt.year

## Calculate the "All Events" column

In [19]:
# group by year
events_year_group = events_df.groupby(['year'])

# get the number of sample events each year
num_events_df = events_year_group['event_id'].count().reset_index()
num_events_df.rename(columns={'event_id': 'All Events'}, inplace=True)
num_events_df.head()

,year,All Events
0,2001,16
1,2002,31
2,2003,1807
3,2004,1109
4,2005,969


## Calculate the "Events with Fish" column

In [20]:
# join events and fish data, merge 'right' to only include events that have fish
wild_with_fish_df = pd.merge(events_df, fish_df, how='right', on=['event_id'])

In [21]:
# get a row for each year-event_id combination
wild_with_fish_year_group = wild_with_fish_df.groupby(['year', 'event_id'])
num_events_with_fish_id_df = wild_with_fish_year_group['fish_id'].count().reset_index()
num_events_with_fish_id_df.head()

,year,event_id,fish_id
0,2001,scfs_1.0,18
1,2001,scfs_10.0,3
2,2001,scfs_11.0,2
3,2001,scfs_12.0,1
4,2001,scfs_13.0,1


In [22]:
# group by year and count the unique event_id for each year
num_events_with_fish_group = num_events_with_fish_id_df.groupby('year')
num_events_with_fish_df = num_events_with_fish_group['event_id'].count().reset_index()
num_events_with_fish_df.rename(columns={'event_id': 'Events with fish'}, inplace=True)
num_events_with_fish_df.head()

,year,Events with fish
0,2001,16
1,2002,31
2,2003,1273
3,2004,799
4,2005,599


## Calculate numbers of fish caught each year

In [23]:
# merge events and fish with all the entries
all_events_fish_df = pd.merge(events_df, fish_df, how='left', on='event_id')
all_events_fish_df.head()

,event_id,sampledate,region,dfozone,sample_site,latitude,longitude,source,source_code,year,...,lep_unknown,cal_cop,cal_chal,cal_motile,cal_unknown,unknown_cop,unknown_chal,unknown_motile,unknown_unknown,lice_protocol
0,bc_1,2003-05-13 00:00:00+00:00,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.6795,Fisheries and Oceans Canada,DFO,2003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lethal
1,bc_1,2003-05-13 00:00:00+00:00,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.6795,Fisheries and Oceans Canada,DFO,2003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lethal
2,bc_1,2003-05-13 00:00:00+00:00,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.6795,Fisheries and Oceans Canada,DFO,2003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lethal
3,bc_1,2003-05-13 00:00:00+00:00,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.6795,Fisheries and Oceans Canada,DFO,2003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lethal
4,bc_1,2003-05-13 00:00:00+00:00,Broughton Archipelago,3_3,Adeane Point,50.71978,-125.6795,Fisheries and Oceans Canada,DFO,2003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lethal


In [24]:
# pivot and count to get counts of each species of fish each year
year_species_df = pd.pivot_table(all_events_fish_df, values='region', index='year', columns=['fish_species'],
                                    aggfunc='count', fill_value=0).reset_index()

year_species_df

fish_species,year,Chinook Salmon,Chum Salmon,Coho Salmon,Other Species,Pacific Herring,Pink Salmon,Sockeye Salmon,Three-Spined Stickleback
0,2001,0,0,0,0,0,268,0,0
1,2002,0,67,0,0,0,497,0,0
2,2003,1011,15484,2909,1614,828,10220,64,2886
3,2004,754,19698,104,1,93,3253,2,1479
4,2005,405,4955,128,1,0,6091,271,45
5,2006,214,14267,170,695,0,11355,9,10
6,2007,345,18770,165,1,0,13907,50,1350
7,2008,521,15785,190,0,0,20920,10,1601
8,2009,126,15699,136,65,6,16680,2,2567
9,2010,137,5991,226,0,0,7381,84,26


## Calculate the total numbers of fish each year

In [25]:
# calculate total number of fish each year
total_fish_df = all_events_fish_df.groupby('year')['fish_id'].count().reset_index()
total_fish_df.head(10)

,year,fish_id
0,2001,268
1,2002,564
2,2003,35016
3,2004,25384
4,2005,11896
5,2006,26720
6,2007,34588
7,2008,39027
8,2009,35281
9,2010,13845


## Merge all the results into the final table

In [26]:
# counts of all the events and events with fish each year
table_1a_df = pd.merge(num_events_df, num_events_with_fish_df, on='year', how='left')
# add in the counts of species each year
table_1a_df = pd.merge(table_1a_df, year_species_df, on='year', how='left')
# add the total number of fish
table_1a_df = pd.merge(table_1a_df, total_fish_df, on='year', how='left')
table_1a_df.head(10)

,year,All Events,Events with fish,Chinook Salmon,Chum Salmon,Coho Salmon,Other Species,Pacific Herring,Pink Salmon,Sockeye Salmon,Three-Spined Stickleback,fish_id
0,2001,16,16,0,0,0,0,0,268,0,0,268
1,2002,31,31,0,67,0,0,0,497,0,0,564
2,2003,1807,1273,1011,15484,2909,1614,828,10220,64,2886,35016
3,2004,1109,799,754,19698,104,1,93,3253,2,1479,25384
4,2005,969,599,405,4955,128,1,0,6091,271,45,11896
5,2006,1098,707,214,14267,170,695,0,11355,9,10,26720
6,2007,1127,943,345,18770,165,1,0,13907,50,1350,34588
7,2008,1214,953,521,15785,190,0,0,20920,10,1601,39027
8,2009,1168,960,126,15699,136,65,6,16680,2,2567,35281
9,2010,732,603,137,5991,226,0,0,7381,84,26,13845


In [27]:
# rename columns nicely for publication
table_1a_df.rename(
    columns={'year': 'Year', 'All Events': 'All events',
             'Chum Salmon': 'Chum', 'Pink Salmon': 'Pink', 'Coho Salmon': 'Coho', 'Chinook Salmon': 'Chinook',
             'Sockeye Salmon': 'Sockeye', 'Three-Spined Stickleback': 'Stickleback', 'Other Species': 'Other species',
             'fish_id': 'All fish'},
    inplace=True
)

# order the columns
col_order = ['Year', 'All events', 'Events with fish','Chum','Pink', 'Coho', 'Chinook', 'Sockeye',
             'Stickleback', 'Pacific Herring', 'Other species', 'All fish']
table_1a_df = table_1a_df.reindex(columns=col_order)
table_1a_df

,Year,All events,Events with fish,Chum,Pink,Coho,Chinook,Sockeye,Stickleback,Pacific Herring,Other species,All fish
0,2001,16,16,0,268,0,0,0,0,0,0,268
1,2002,31,31,67,497,0,0,0,0,0,0,564
2,2003,1807,1273,15484,10220,2909,1011,64,2886,828,1614,35016
3,2004,1109,799,19698,3253,104,754,2,1479,93,1,25384
4,2005,969,599,4955,6091,128,405,271,45,0,1,11896
5,2006,1098,707,14267,11355,170,214,9,10,0,695,26720
6,2007,1127,943,18770,13907,165,345,50,1350,0,1,34588
7,2008,1214,953,15785,20920,190,521,10,1601,0,0,39027
8,2009,1168,960,15699,16680,136,126,2,2567,6,65,35281
9,2010,732,603,5991,7381,226,137,84,26,0,0,13845


In [28]:
# export the table to a CSV for final formatting
table_1a_df.to_csv(OUTPUT_DIR / "Table_1a.csv", index=False)